# 🔍 Exploration du Dataset MNIST

Bienvenue dans ce deuxième notebook ! Aujourd'hui, nous allons découvrir le dataset MNIST.

## 📚 Qu'est-ce que MNIST ?

**MNIST** (Modified National Institute of Standards and Technology) est :
- 📊 Un dataset de **70,000 images** de chiffres manuscrits (0-9)
- ✍️ Des chiffres écrits à la main par des personnes réelles
- 🎯 Le "Hello World" du deep learning
- 🏆 Un benchmark classique pour tester les algorithmes

### Structure du dataset :
- **60,000 images** pour l'entraînement (train set)
- **10,000 images** pour le test (test set)
- Chaque image : **28×28 pixels** en niveaux de gris
- Valeurs des pixels : **0 (noir) à 255 (blanc)**

---

In [ ]:
# Imports nécessaires
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Ajouter le dossier parent au path pour importer nos modules
sys.path.append(str(Path.cwd().parent))

# Configuration pour de beaux graphiques
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
sns.set_palette("husl")

print("✅ Imports réussis !")

## 1️⃣ Chargement des Données

Commençons par charger les données MNIST. Nous allons utiliser le module utilitaire que nous avons préparé.

In [ ]:
# Importer notre fonction de chargement
from src.utils import load_mnist_data

# Charger les données
print("⏳ Chargement des données MNIST...")
X_train, y_train, X_test, y_test = load_mnist_data()

print("\n✅ Données chargées avec succès !\n")
print("=" * 50)
print("📊 INFORMATIONS SUR LE DATASET")
print("=" * 50)
print(f"\n🔹 Training set:")
print(f"   - Images (X_train): {X_train.shape}")
print(f"   - Labels (y_train): {y_train.shape}")
print(f"\n🔹 Test set:")
print(f"   - Images (X_test): {X_test.shape}")
print(f"   - Labels (y_test): {y_test.shape}")
print(f"\n🔹 Détails:")
print(f"   - Chaque image: 28×28 = {28*28} pixels")
print(f"   - Valeurs des pixels: {X_train.min():.2f} (noir) à {X_train.max():.2f} (blanc)")
print(f"   - Type de données: {X_train.dtype}")

### 💡 Comprendre les dimensions

- **X_train.shape = (60000, 784)** signifie :
  - 60,000 exemples d'entraînement
  - Chaque exemple a 784 features (28×28 pixels aplatis)

- **y_train.shape = (60000,)** signifie :
  - 60,000 labels correspondants (les vrais chiffres)
  
Chaque image 28×28 est "aplatie" en un vecteur de 784 valeurs pour faciliter le calcul.

## 2️⃣ Visualisons Quelques Exemples

Regardons à quoi ressemblent ces chiffres manuscrits !

In [ ]:
def visualize_samples(X, y, n_samples=25):
    """
    Affiche un échantillon d'images du dataset
    """
    # Sélectionner des échantillons aléatoires
    indices = np.random.choice(len(X), n_samples, replace=False)
    
    # Créer une grille
    n_rows = 5
    n_cols = 5
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 12))
    fig.suptitle('🖼️ Échantillons du Dataset MNIST', fontsize=18, fontweight='bold', y=0.995)
    
    for idx, ax in enumerate(axes.flat):
        # Récupérer l'image et la reformater en 28x28
        image = X[indices[idx]].reshape(28, 28)
        label = y[indices[idx]]
        
        # Afficher l'image
        ax.imshow(image, cmap='gray_r')  # gray_r = inverse (noir sur blanc)
        ax.set_title(f'Label: {label}', fontsize=14, fontweight='bold', pad=10)
        ax.axis('off')
        
        # Ajouter une bordure colorée selon le chiffre
        colors = plt.cm.tab10(label)
        for spine in ax.spines.values():
            spine.set_edgecolor(colors)
            spine.set_linewidth(3)
            spine.set_visible(True)
    
    plt.tight_layout()
    plt.show()

# Afficher des exemples
visualize_samples(X_train, y_train)

### 🎨 Observation

On peut voir que :
- ✍️ Les écritures sont **très variées** (chacun écrit différemment)
- 📏 Certains chiffres sont **penchés**
- 📐 Les **tailles** varient
- 🎭 Certains sont plus **épais** ou **fins**

C'est justement cette variabilité qui rend le problème intéressant !

## 3️⃣ Distribution des Classes

Vérifions si le dataset est **équilibré** (même nombre d'exemples pour chaque chiffre).

In [ ]:
def plot_class_distribution(y_train, y_test):
    """
    Affiche la distribution des classes dans les datasets train et test
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Distribution du train set
    unique_train, counts_train = np.unique(y_train, return_counts=True)
    bars1 = ax1.bar(unique_train, counts_train, color=plt.cm.Spectral(np.linspace(0, 1, 10)), 
                    edgecolor='black', linewidth=2, alpha=0.8)
    ax1.set_xlabel('Chiffre', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Nombre d\'exemples', fontsize=14, fontweight='bold')
    ax1.set_title('📊 Distribution - Training Set', fontsize=16, fontweight='bold')
    ax1.set_xticks(unique_train)
    ax1.grid(axis='y', alpha=0.3)
    
    # Ajouter les valeurs au-dessus des barres
    for bar, count in zip(bars1, counts_train):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(count)}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Distribution du test set
    unique_test, counts_test = np.unique(y_test, return_counts=True)
    bars2 = ax2.bar(unique_test, counts_test, color=plt.cm.Spectral(np.linspace(0, 1, 10)),
                    edgecolor='black', linewidth=2, alpha=0.8)
    ax2.set_xlabel('Chiffre', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Nombre d\'exemples', fontsize=14, fontweight='bold')
    ax2.set_title('📊 Distribution - Test Set', fontsize=16, fontweight='bold')
    ax2.set_xticks(unique_test)
    ax2.grid(axis='y', alpha=0.3)
    
    # Ajouter les valeurs au-dessus des barres
    for bar, count in zip(bars2, counts_test):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(count)}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques
    print("\n" + "="*60)
    print("📈 STATISTIQUES DE DISTRIBUTION")
    print("="*60)
    print(f"\n🔹 Training Set:")
    print(f"   - Min: {counts_train.min()} exemples (chiffre {unique_train[counts_train.argmin()]})")
    print(f"   - Max: {counts_train.max()} exemples (chiffre {unique_train[counts_train.argmax()]})")
    print(f"   - Moyenne: {counts_train.mean():.0f} exemples par classe")
    print(f"   - Écart-type: {counts_train.std():.2f}")
    
    print(f"\n🔹 Test Set:")
    print(f"   - Min: {counts_test.min()} exemples (chiffre {unique_test[counts_test.argmin()]})")
    print(f"   - Max: {counts_test.max()} exemples (chiffre {unique_test[counts_test.argmax()]})")
    print(f"   - Moyenne: {counts_test.mean():.0f} exemples par classe")
    print(f"   - Écart-type: {counts_test.std():.2f}")
    
    # Évaluation de l'équilibre
    balance_train = (counts_train.max() - counts_train.min()) / counts_train.mean() * 100
    balance_test = (counts_test.max() - counts_test.min()) / counts_test.mean() * 100
    
    print(f"\n🔹 Équilibre du dataset:")
    print(f"   - Train: {balance_train:.2f}% de variation")
    print(f"   - Test: {balance_test:.2f}% de variation")
    
    if balance_train < 10 and balance_test < 10:
        print(f"   ✅ Dataset bien équilibré !")
    else:
        print(f"   ⚠️ Dataset légèrement déséquilibré")

plot_class_distribution(y_train, y_test)

## 4️⃣ Analyse d'une Image en Détail

Regardons de plus près comment une image est représentée numériquement.

In [ ]:
def analyze_single_image(X, y, index=0):
    """
    Analyse détaillée d'une seule image
    """
    # Sélectionner une image
    image_flat = X[index]
    image_2d = image_flat.reshape(28, 28)
    label = y[index]
    
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # 1. Image originale (grande)
    ax1 = fig.add_subplot(gs[0:2, 0:2])
    im1 = ax1.imshow(image_2d, cmap='gray_r', interpolation='nearest')
    ax1.set_title(f'🖼️ Image du chiffre: {label}', fontsize=16, fontweight='bold', pad=15)
    ax1.axis('off')
    plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04, label='Intensité du pixel')
    
    # 2. Zoom sur une partie (coin supérieur gauche)
    ax2 = fig.add_subplot(gs[0, 2])
    zoom_size = 8
    zoom_region = image_2d[:zoom_size, :zoom_size]
    im2 = ax2.imshow(zoom_region, cmap='gray_r', interpolation='nearest')
    ax2.set_title('🔍 Zoom (8×8 pixels)', fontsize=12, fontweight='bold')
    
    # Afficher les valeurs des pixels
    for i in range(zoom_size):
        for j in range(zoom_size):
            text = ax2.text(j, i, f'{zoom_region[i, j]:.2f}',
                          ha="center", va="center", color="red", fontsize=7, fontweight='bold')
    ax2.set_xticks(range(zoom_size))
    ax2.set_yticks(range(zoom_size))
    ax2.grid(True, color='blue', linewidth=1)
    
    # 3. Histogramme des valeurs de pixels
    ax3 = fig.add_subplot(gs[1, 2])
    ax3.hist(image_flat, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
    ax3.set_xlabel('Valeur du pixel', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Fréquence', fontsize=11, fontweight='bold')
    ax3.set_title('📊 Distribution des pixels', fontsize=12, fontweight='bold')
    ax3.grid(axis='y', alpha=0.3)
    
    # 4. Statistiques de l'image
    ax4 = fig.add_subplot(gs[2, :])
    ax4.axis('off')
    
    stats_text = f"""
    📈 STATISTIQUES DE L'IMAGE
    {'='*80}
    
    🔹 Dimensions:
       • Forme originale: 28×28 pixels
       • Forme aplatie: {image_flat.shape[0]} valeurs (28×28 = 784)
    
    🔹 Valeurs des pixels:
       • Minimum: {image_flat.min():.4f}  (noir complet)
       • Maximum: {image_flat.max():.4f}  (blanc complet)
       • Moyenne: {image_flat.mean():.4f}
       • Écart-type: {image_flat.std():.4f}
       • Médiane: {np.median(image_flat):.4f}
    
    🔹 Composition:
       • Pixels noirs (= 0): {np.sum(image_flat == 0)} pixels ({np.sum(image_flat == 0)/784*100:.1f}%)
       • Pixels non-noirs (> 0): {np.sum(image_flat > 0)} pixels ({np.sum(image_flat > 0)/784*100:.1f}%)
       • Pixels très clairs (> 0.5): {np.sum(image_flat > 0.5)} pixels ({np.sum(image_flat > 0.5)/784*100:.1f}%)
    
    💡 Note: Les valeurs sont normalisées entre 0 (noir) et 1 (blanc)
    """
    
    ax4.text(0.05, 0.5, stats_text, transform=ax4.transAxes,
            fontsize=11, verticalalignment='center', family='monospace',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8, pad=1))
    
    plt.suptitle('🔬 Analyse Détaillée d\'une Image MNIST', 
                fontsize=18, fontweight='bold', y=0.98)
    
    plt.show()

# Analyser une image aléatoire
random_index = np.random.randint(0, len(X_train))
analyze_single_image(X_train, y_train, random_index)

## 5️⃣ Moyenne de Chaque Chiffre

Calculons l'image "moyenne" de chaque chiffre pour voir les patterns typiques.

In [ ]:
def plot_mean_images(X, y):
    """
    Affiche l'image moyenne pour chaque classe
    """
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle('📊 Image Moyenne de Chaque Chiffre', fontsize=18, fontweight='bold')
    
    for digit in range(10):
        # Sélectionner toutes les images de ce chiffre
        digit_images = X[y == digit]
        
        # Calculer la moyenne
        mean_image = digit_images.mean(axis=0).reshape(28, 28)
        
        # Afficher
        ax = axes[digit // 5, digit % 5]
        im = ax.imshow(mean_image, cmap='gray_r')
        ax.set_title(f'Chiffre {digit}\n({len(digit_images)} exemples)', 
                    fontsize=13, fontweight='bold')
        ax.axis('off')
        
        # Ajouter une bordure colorée
        for spine in ax.spines.values():
            spine.set_edgecolor(plt.cm.tab10(digit))
            spine.set_linewidth(4)
            spine.set_visible(True)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Observation:")
    print("   Les images moyennes montrent le 'prototype' de chaque chiffre.")
    print("   On peut voir les formes typiques et les zones communes à tous les exemples.")

plot_mean_images(X_train, y_train)

## 6️⃣ Variabilité Intra-Classe

Regardons à quel point les écritures d'un même chiffre peuvent varier.

In [ ]:
def plot_digit_variations(X, y, digit, n_samples=20):
    """
    Affiche plusieurs exemples d'un même chiffre pour montrer la variabilité
    """
    # Sélectionner des exemples de ce chiffre
    digit_indices = np.where(y == digit)[0]
    selected_indices = np.random.choice(digit_indices, n_samples, replace=False)
    
    # Créer la grille
    fig, axes = plt.subplots(4, 5, figsize=(14, 11))
    fig.suptitle(f'✍️ Variabilité du Chiffre "{digit}" - {n_samples} Exemples Différents', 
                fontsize=18, fontweight='bold')
    
    for idx, ax in enumerate(axes.flat):
        image = X[selected_indices[idx]].reshape(28, 28)
        ax.imshow(image, cmap='gray_r')
        ax.axis('off')
        ax.set_title(f'Exemple {idx+1}', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n🎭 Observations sur le chiffre {digit}:")
    print("   • Les écritures varient énormément d'une personne à l'autre")
    print("   • Certains sont penchés, d'autres droits")
    print("   • L'épaisseur du trait varie")
    print("   • La position dans l'image change légèrement")
    print("   \n   ➡️ C'est cette variabilité que le réseau doit apprendre à gérer !")

# Choisir un chiffre aléatoire
random_digit = np.random.randint(0, 10)
plot_digit_variations(X_train, y_train, random_digit)

## 7️⃣ Cas Difficiles : Chiffres Ambigus

Certains chiffres sont difficiles à distinguer, même pour un humain !

In [ ]:
def find_confusing_pairs(X, y):
    """
    Trouve des exemples de chiffres qui pourraient être confondus
    """
    # Paires de chiffres souvent confondus
    confusing_pairs = [
        (3, 8),  # 3 et 8 peuvent se ressembler
        (4, 9),  # 4 et 9 peuvent être similaires
        (1, 7),  # 1 et 7 selon l'écriture
        (5, 6),  # 5 et 6 peuvent être proches
    ]
    
    fig, axes = plt.subplots(len(confusing_pairs), 6, figsize=(14, 10))
    fig.suptitle('🤔 Paires de Chiffres Potentiellement Confondables', 
                fontsize=18, fontweight='bold')
    
    for pair_idx, (digit1, digit2) in enumerate(confusing_pairs):
        # Trouver des exemples de chaque chiffre
        indices1 = np.where(y == digit1)[0]
        indices2 = np.where(y == digit2)[0]
        
        # Sélectionner 3 exemples de chaque
        samples1 = np.random.choice(indices1, 3, replace=False)
        samples2 = np.random.choice(indices2, 3, replace=False)
        
        # Afficher les exemples du premier chiffre
        for i in range(3):
            ax = axes[pair_idx, i]
            image = X[samples1[i]].reshape(28, 28)
            ax.imshow(image, cmap='gray_r')
            ax.axis('off')
            if i == 1:
                ax.set_title(f'Chiffre {digit1}', fontsize=12, fontweight='bold', color='blue')
        
        # Afficher les exemples du second chiffre
        for i in range(3):
            ax = axes[pair_idx, i+3]
            image = X[samples2[i]].reshape(28, 28)
            ax.imshow(image, cmap='gray_r')
            ax.axis('off')
            if i == 1:
                ax.set_title(f'Chiffre {digit2}', fontsize=12, fontweight='bold', color='red')
    
    plt.tight_layout()
    plt.show()
    
    print("\n💭 Ces paires peuvent être difficiles à distinguer car:")
    print("   • 3 et 8: Formes arrondies similaires")
    print("   • 4 et 9: Partie supérieure peut être similaire")
    print("   • 1 et 7: Selon le style d'écriture")
    print("   • 5 et 6: Courbes du bas similaires")
    print("   \n   ➡️ Un bon réseau doit apprendre les différences subtiles !")

find_confusing_pairs(X_train, y_train)

## 8️⃣ Analyse de la Corrélation entre Pixels

Regardons quels pixels ont tendance à être activés ensemble.

In [ ]:
def analyze_pixel_statistics(X, y, digit):
    """
    Analyse les statistiques des pixels pour un chiffre donné
    """
    # Sélectionner toutes les images de ce chiffre
    digit_images = X[y == digit]
    
    # Calculer moyenne et écart-type
    mean_image = digit_images.mean(axis=0).reshape(28, 28)
    std_image = digit_images.std(axis=0).reshape(28, 28)
    
    # Visualiser
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    fig.suptitle(f'📊 Analyse Statistique du Chiffre "{digit}"', 
                fontsize=18, fontweight='bold')
    
    # Moyenne
    im1 = axes[0].imshow(mean_image, cmap='gray_r')
    axes[0].set_title('Valeur Moyenne\npar Pixel', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    plt.colorbar(im1, ax=axes[0], fraction=0.046, label='Intensité moyenne')
    
    # Écart-type
    im2 = axes[1].imshow(std_image, cmap='hot')
    axes[1].set_title('Écart-type\npar Pixel', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    plt.colorbar(im2, ax=axes[1], fraction=0.046, label='Variabilité')
    
    # Variance (écart-type au carré)
    variance_image = std_image ** 2
    im3 = axes[2].imshow(variance_image, cmap='viridis')
    axes[2].set_title('Variance\npar Pixel', fontsize=14, fontweight='bold')
    axes[2].axis('off')
    plt.colorbar(im3, ax=axes[2], fraction=0.046, label='Variance')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n🔍 Interprétation pour le chiffre {digit}:")
    print("\n   📊 Valeur Moyenne:")
    print("      • Montre où le trait est généralement présent")
    print("      • Les zones claires = présence constante du chiffre")
    print("\n   🎭 Écart-type:")
    print("      • Zones colorées = grande variabilité entre les exemples")
    print("      • Zones sombres = tous les exemples sont similaires ici")
    print("\n   💡 Variance:")
    print("      • Mesure alternative de la variabilité")
    print("      • Utile pour identifier les pixels les plus 'informatifs'")

# Analyser un chiffre aléatoire
random_digit = np.random.randint(0, 10)
analyze_pixel_statistics(X_train, y_train, random_digit)

## 9️⃣ Préparation pour l'Entraînement

Avant d'entraîner un réseau, vérifions que nos données sont bien formatées.

In [ ]:
def check_data_quality(X_train, y_train, X_test, y_test):
    """
    Vérifie la qualité et le format des données
    """
    print("\n" + "="*70)
    print("✅ VÉRIFICATION DE LA QUALITÉ DES DONNÉES")
    print("="*70)
    
    checks = []
    
    # Vérification 1: Pas de valeurs manquantes
    no_nan_train = not np.any(np.isnan(X_train))
    no_nan_test = not np.any(np.isnan(X_test))
    checks.append(("Pas de valeurs manquantes (NaN)", no_nan_train and no_nan_test))
    
    # Vérification 2: Valeurs normalisées entre 0 et 1
    normalized_train = (X_train.min() >= 0) and (X_train.max() <= 1)
    normalized_test = (X_test.min() >= 0) and (X_test.max() <= 1)
    checks.append(("Valeurs normalisées [0, 1]", normalized_train and normalized_test))
    
    # Vérification 3: Bonnes dimensions
    correct_shape_train = X_train.shape == (60000, 784)
    correct_shape_test = X_test.shape == (10000, 784)
    checks.append(("Dimensions correctes", correct_shape_train and correct_shape_test))
    
    # Vérification 4: Labels valides (0-9)
    valid_labels_train = np.all((y_train >= 0) & (y_train <= 9))
    valid_labels_test = np.all((y_test >= 0) & (y_test <= 9))
    checks.append(("Labels valides (0-9)", valid_labels_train and valid_labels_test))
    
    # Vérification 5: Toutes les classes présentes
    all_classes_train = len(np.unique(y_train)) == 10
    all_classes_test = len(np.unique(y_test)) == 10
    checks.append(("Toutes les classes présentes (0-9)", all_classes_train and all_classes_test))
    
    # Vérification 6: Pas d'images totalement noires
    no_empty_train = not np.any(X_train.sum(axis=1) == 0)
    no_empty_test = not np.any(X_test.sum(axis=1) == 0)
    checks.append(("Pas d'images vides", no_empty_train and no_empty_test))
    
    # Afficher les résultats
    print("\n🔍 Résultats des vérifications:\n")
    for check_name, passed in checks:
        status = "✅" if passed else "❌"
        print(f"   {status} {check_name}")
    
    # Résumé
    all_passed = all([passed for _, passed in checks])
    
    print("\n" + "="*70)
    if all_passed:
        print("\n🎉 FÉLICITATIONS ! Toutes les vérifications sont passées.")
        print("   Les données sont prêtes pour l'entraînement !")
    else:
        print("\n⚠️ ATTENTION ! Certaines vérifications ont échoué.")
        print("   Vérifiez les données avant de continuer.")
    print("\n" + "="*70)
    
    # Informations supplémentaires
    print("\n📋 Informations supplémentaires:\n")
    print(f"   • Type de données: {X_train.dtype}")
    print(f"   • Mémoire utilisée (train): {X_train.nbytes / 1024**2:.2f} MB")
    print(f"   • Mémoire utilisée (test): {X_test.nbytes / 1024**2:.2f} MB")
    print(f"   • Total: {(X_train.nbytes + X_test.nbytes) / 1024**2:.2f} MB")
    
    return all_passed

# Effectuer les vérifications
data_ready = check_data_quality(X_train, y_train, X_test, y_test)

## 🎯 Récapitulatif

Dans ce notebook, nous avons exploré en profondeur le dataset MNIST :

### ✅ Ce que nous avons appris

1. **Structure du dataset** :
   - 60,000 images d'entraînement + 10,000 images de test
   - Images 28×28 pixels normalisées entre 0 et 1
   - 10 classes (chiffres 0-9)

2. **Visualisations importantes** :
   - Échantillons aléatoires du dataset
   - Distribution équilibrée des classes
   - Analyse détaillée d'une image
   - Images moyennes par classe

3. **Défis du dataset** :
   - Grande variabilité intra-classe (même chiffre, écritures différentes)
   - Certains chiffres peuvent être ambigus
   - Le réseau doit apprendre des features robustes

4. **Qualité des données** :
   - Vérifications de cohérence
   - Données prêtes pour l'entraînement

### 🎓 Points clés à retenir

- 📊 **Toujours explorer ses données** avant de commencer l'entraînement
- 👀 **Comprendre la variabilité** aide à concevoir le bon réseau
- ⚖️ **Vérifier l'équilibre** des classes pour éviter les biais
- 🔍 **Identifier les cas difficiles** pour mieux évaluer le modèle

---

### 🚀 Prochaines Étapes

Maintenant que nous connaissons bien nos données, passons au notebook suivant :

**`02_forward_propagation.ipynb`** - Comprendre comment un réseau fait des prédictions !

---

### 💪 Exercices suggérés

Avant de passer au suivant, essaie ces exercices :

1. Trouve les 5 images du chiffre "1" les plus différentes
2. Calcule le chiffre qui a la plus grande variance moyenne de pixels
3. Crée une visualisation montrant les 10 images les plus "typiques" de chaque classe
4. Identifie des exemples qui pourraient être mal étiquetés (ambigus même pour un humain)

Bonne exploration ! 🎉